In [1]:
import fastai

In [2]:
fastai.__version__

'1.0.42'

In [3]:
from fastai.text import * 

## Train/valid split
Since there are only 19 positive examples, put half in validation set.

In [10]:
import glob

In [34]:
random.seed(325732)  # reproducible split
for filename in glob.glob("D:/Notebooks/email_urgent/data/v1/*/*.txt"):
    p = pathlib.Path(filename)
    trainvalid = "train" if random.randint(0, 1) else "valid"
    newpath = pathlib.Path(*(p.parts[:-2] + (trainvalid,) + p.parts[-2:]))
    #print(f"{p} -> {newpath}")
    # workaround for windows paths longer than ~260 chars
    shutil.copy(filename, u'\\\\?\\' + str(newpath))

In [38]:
path = Path("\\\\?\\D:/Notebooks/email_urgent/data/v1/")
path

WindowsPath('//?/D:/Notebooks/email_urgent/data/v1')

In [39]:
data_lm = TextLMDataBunch.from_folder(path)

In [40]:
# small batch size for sample. Change this when there is more data!
data_clas = TextClasDataBunch.from_folder(path, vocab=data_lm.train_ds.vocab, bs=16)

In [41]:
# This may not be using a pretrained model
learn = language_model_learner(data_lm, drop_mult=0.5)

In [42]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,6.059590,5.619079,0.081715


In [43]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,5.709442,5.575981,0.081715


In [44]:
learn.save_encoder('ft_enc')

In [57]:
learn = text_classifier_learner(data_clas, drop_mult=0.5, metrics=[Precision(), Recall(), accuracy])

In [58]:
learn.load_encoder('ft_enc')

Remember classes are unbalanced!

In [59]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,precision,recall,accuracy
1,0.557430,0.380856,nan,0.000000,0.962025


In [60]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,precision,recall,accuracy
1,0.412593,0.220159,nan,0.000000,0.962025
